# CAPSTONE 3. Predicting Major Cryptocurrencies Prices
## Modeling

In this notebook we will continue to implement different times series forecasting algorithms in order to predict Bitcoin price. In particular, we will build three models:
<ul>
    <i>Exponential Smoothing</i> - a time series forecasting method for univariate data, which can be used as an alternative for ARIMA family of methods<br>
    <i>Prophet</i> - a time-series forecasting model developped by Facebook<br>
    <i>PyCaret</i> - a Python version of the Caret machine learning package in R<br>
</ul>
As always, we will train the models on the training set and evaluate them on the test set. The train and test sets will be the same that we used in the previous notebook.<br>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import warnings
import os
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from fbprophet import Prophet
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
from joblib import Parallel
from joblib import delayed
from multiprocessing import cpu_count
from math import sqrt

warnings.filterwarnings('ignore')

#setting figure's default size
sns.set(rc={'figure.figsize':(12,5)})
plt.rcParams['figure.figsize'] = (12,5)

sns.set_style('whitegrid')

pd.options.display.float_format = "{:.3f}".format

First, let's import the data we saved in the previous step. We will use it for our analysis.

In [ ]:
df = pd.read_pickle('../PTDD/df.pkl')
train = np.load('../PTDD/train_norm.npy')
test = np.load('../PTDD/test_norm.npy')
btc = pd.read_pickle('../EDA/btc.pkl')
btc_price = btc[['Date', 'Close']].sort_values(by='Date', ascending=True)

In [ ]:
print(f'Train set observations: {len(train)}')
print(f'Test set observations: {len(test)}')

And take a look once again at BTC price time series.

In [ ]:
sns.lineplot(data=btc, x='Date', y='Close', color='Gold')
plt.title(f'August 2015 - January 2021 BTC price')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.show();

### Algorithm 1 - Exponential Smoothing

Exponential Smoothing (ES) is a time series forecasting algorithm which can be seen as an analog for Box-Jenkins ARIMA family of methonds. Both of them make a prediction as a weighted sum of previous observations. The main difference is that Exponential Smoothing uses an exponentially decreasing weight for previous observations, using parameter named "alpha". The bigger alpha is, the less important the older observations become for the prediction. If alpha is small, on the other hand, then the model pays more attention to the older obserbations.

There are three types of Exponential Smoothing:<br>
<ol>
    1. Single Exponential Smoothing - does not take into account trend and seasonalirt<br>
    2. Double Exponential Smoothing - takes trend into account<br>
    3. Triple Exponential Smoothing - takes into account both trend and seasonality<br>
</ol>

We know from the previous step that our time series is not stationary. We are not sure if it has clear seasonality, so we will build Double ES model first and than compare it to Triple ES, to see how seasonality will affect the model's performance.<br>
It is also important to understand what kind of trend our time series has - additive or multiplicative (we will need to specify it as a hyperparameter). On the graph we see that even though in the last couple month there is an exponential growth, for 5 years before that the general trend seems to be linear, so we will assume the additive trend as our baseline option.

In [ ]:
# train-test split
train_size = int(len(btc_price) * 0.985)
train, test = btc_price[0:train_size], btc_price[train_size:len(btc_price)]
print(f'Total Observations: {(len(btc_price))}')
print(f'Training Observations: {(len(train))}')
print(f'Testing Observations: {(len(test))}')

In [ ]:
train.set_index('Date', inplace=True)

In [ ]:
# initializing a model, fitting on the train set and predicting for the next 30 observations (equivalent to the test set length)
des_model = ExponentialSmoothing(train, trend='add', damped=False)
des_model_fit = des_model.fit()
des_model_pred = des_model_fit.forecast(30)

In [ ]:
predictions_test = [x for x in des_model_pred] # List with predicted test values
observations_test = [x for x in test['Close']] # List with expected test values

In [ ]:
r2 = r2_score(observations_test, predictions_test)
print(f'Test set r-squared for Double ES (no tuning) is: {round(r2, 2)}')
mepe = mean_absolute_percentage_error(observations_test, predictions_test)
print(f'Test set MAPE for Double ES (no tuning) is: {round(mepe, 2) * 100}%')

In [ ]:
plt.plot(observations_test, label='Expected')
plt.plot(predictions_test, label='Predicted', color='red', alpha=0.5)
legend = plt.legend(loc='upper center', shadow=True, fontsize='x-large')
plt.title("Train set Predicted vs Expected")
plt.show()

As we can see, the out-of-box Exponential Smoothing model didn't work pretty much at all. We have a negative r-squared which means the model fits the data even worse than just a straight line. The Mean Absolute Percentage Error is also very high (28% against 2% of ARIMA(2,1,1))

In order to make this model work better, we will perform hyperparemeters tuning via GridSearch.

### Tuning Exponential Smoothing Model

Unfortunately, there is no analog of Scikit-Learn GridSearchCV() function for time series. This means we will have to write a few user-defined functions in order to implement hyperparameter tuning process. We will use Walk Forward Validation approach which allows for a model to be updated every time step it recieves new data.

In [ ]:
# def ES_forecast(history, config):
#     '''This function performs one-ster Exponential Smoothing forecast.
#        t - trend, d - dampening, s - seasonality, p - period,
#        b - Box-Cox transform (bool), r - remove bias (bool)'''
#     t,d,s,p,b,r = config
#     history = np.array(history)
#     model = ExponentialSmoothing(history, trend=t, dampen=d, seasonal=s, seasonal_periods=p)
#     model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
#     y_pred = model_fit.predict(len(history), len(history))
#     return y_pred

Then we will write a function that splits the time series into train and test sets.

In [ ]:
# def train_test_split(ts, test_size):
#     '''This function splits the given time series into train and test sets'''
#     return ts[:-test_size], ts[-test_size:]

Then we will need to measure the errors for our model. We will use both r-squared and Mean Absolute Percentage Error.

In [ ]:
# def r2(expected, predicted):
#     '''This function returns r-squared'''
#     r2 = r2_score(expected, predicted)
#     return round(r2, 2)
# def mape(expected, predicted):
#     '''This function returns MAPE'''
#     mape = mean_absolute_percentage_error(expected, predicted)
#     return round(mape, 2) * 100+'%'

Then we will need a function that will perform one-step validation and return the errors.

In [ ]:
# def walk_forward_validaion(ts, test_size, config):
#     '''This function:
#         1. Splits the time series into train and test sets
#         2. Enumerated the number of train set observations
#         3. Fits a model on each train set observation
#         4. Adds predicted value to predictions list
#         5. Adds expected values to history list
#         6. Calculates r-squared and MAPE for expected vs. predicted values'''
#     prdeictions = []
#     train, test = train_test_split(ts, test_size)
#     history = [x for x in train]
#     for i in range(len(test)):
#         y_pred = ES_forecast(history, config)
#         predictions.append(y_pred)
#         history.append(test[i])
#     r2 = r2(test, predictions)
# #     print(f'r-squared for Exponential Smoothing (config: {config}) is: {r2}')
#     mape = mape(test, predictions)
# #     print(f'Mean Absolute Percentage Error for Exponential Smoothing (config: {config}) is: {mape}')
#     return (r2, mape)

Now we will write one more function. In case there is an error it will retrun None. If everything runs smoothly it will return the results of model evaluation.

In [ ]:
# def eval_model(ts, test_size, config, defug=False):
#     '''This function performs walk forward validation'''
#     result=None
#     conf = str(config)
#     if debug:
#         result = walk_forward_validation(ts, test_size, config)
#     else:
#         error = None
#     if result is not None:
#         print(print(' > Model[%s] %.3f' % (conf, result)))
#     return(conf, result)

Grid search is a brute force method, so it requires a lot of computational power and time. We can reduce this time by parallelizing the grid search and making use of all of your machine's CPUs.

In [ ]:
# def grid_search_NO(ts, config_list, test_size, parallel=True):
#     '''This function performs grid search over different mode parameters.
#         By default it uses parallel computations.
#         It returns results of the different configurations evaluation.'''
#     scores = None
#     if parallel:
#         parallelizer = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
#         tasks = (delayed(eval_model)(ts, test_size, config) for config in config_list)
#         scores = parallelizer(tasks)
#     else:
#         scores = [eval_model(ts, test_size, config) for config in config_list]
#     # removing empty results
#     scores = [res for res in scores if res[1] != None]
#     scores.sort(key=lambda tup:tup[1])
#     return scores

Now we will need to create a list of model parameters configurations which we will use in our grid search.

In [ ]:
# def ES_configs(seasonal=[None]):
#     '''This function returns a list of different model configurations
#         which are used in grid search'''
#     configs = []
#     # trend parameters
#     t_params = ['add', 'mul', None]
#     # dampening parameters
#     d_params = [True, False]
#     # seasonality parameters)
#     s_params = ['add', 'mul', None]
#     # period parameters (default=None)
#     p_params = seasonal
#     # Box-Cox parameters
#     b_params = [True, False]
#     # remove bias parameters
#     r_params = [True, False]
#     for t in t_params:
#         for d in d_params:
#             for s in s_params:
#                 for p in p_params:
#                     for b in b_params:
#                         for r in r_params:
#                             config = [t,d,s,p,b,r]
#                             configs.append(config)
#     return configs

Finally, we can implement the grid search.

In [ ]:
btc_price_for_func = btc_price.set_index('Date')

In [ ]:
# grid search ets models for monthly car sales
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from pandas import read_csv
from numpy import array

# one-step Holt Winter’s Exponential Smoothing forecast
def exp_smoothing_forecast(history, config):
	t,d,s,p,b,r = config
	# define model
	history = array(history)
	model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
	# fit model
	model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = exp_smoothing_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of exponential smoothing configs to try
def exp_smoothing_configs(seasonal=[None]):
	models = list()
	# define config lists
	t_params = ['add', 'mul', None]
	d_params = [True, False]
	s_params = ['add', 'mul', None]
	p_params = seasonal
	b_params = [True, False]
	r_params = [True, False]
	# create config instances
	for t in t_params:
		for d in d_params:
			for s in s_params:
				for p in p_params:
					for b in b_params:
						for r in r_params:
							cfg = [t,d,s,p,b,r]
							models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	series = read_csv('monthly-car-sales.csv', header=0, index_col=0)
	data = series.values
	# data split
	n_test = 12
	# model configs
	cfg_list = exp_smoothing_configs(seasonal=[0,6,12])
	# grid search
	scores = grid_search(data[:,0], cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)